# Interaktivni prikaz

Notebook obuhvaća isto gradivo kao 01, ali s interaktivnim prikazom, bez dodatnih komentara.
Da bi ispitali djelovanje s drugim procesom, promijenite funkciju get_model.

In [7]:
# priprema okruženja
%matplotlib widget
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
plt.rcParams.update({'figure.max_open_warning': 0})
import control

import ipywidgets as ipw

In [12]:
s=control.tf([1, 0], [1])
def get_model():
    return 1/((s*s+2*s+2)*(s+3))

print("Gp =")
get_model()

Gp =



          1
---------------------
s^3 + 5 s^2 + 8 s + 6

In [13]:
slider=ipw.FloatSlider(description='Kr', value=1, min=1, max=100, step=0.1)

#interaktivni prikaz - bode
def replot_bode(data=None):
    Kp = slider.value
    
    Gp = get_model()
    Gr=Kp
    Gz = control.feedback(Gr*Gp)
    
    T = np.linspace(0, 10, 200)
    tout, yout = control.step_response(Gz, T)
    w_min = 1e-2
    w_max = 1e1
    ws = np.exp(np.linspace(np.log(w_min), np.log(w_max), 200))
    mag, phase, omega=control.bode(Gr*Gp, omega=ws, Plot=False)

    # mark 180 deg points
    i_freq = np.argmax(phase<-np.pi)
    
    ax1.clear()
    ax1.set_title('Odziv Gz na skokovitu pobudu, polovi')
    ax1.plot(tout, yout)
    ax1.grid()

    ax2.clear()
    poles, zeros = control.pzmap(Gz, Plot=False)
    if len(poles) > 0:
        ax2.scatter(np.real(poles), np.imag(poles), s=50, marker='x',
                   facecolors='k')
    if len(zeros) > 0:
        ax2.scatter(np.real(zeros), np.imag(zeros), s=50, marker='o',
                   facecolors='none', edgecolors='k')    
        
    # Add axes
    #ax2.axhline(y=0, color='red')
    ax2.axvline(x=0, color='red')
    ax2.set_xlabel('Re')
    ax2.set_ylabel('Im')
    
    ax2.set_xlim([-10,10])
    ax2.grid()
    
    ax3.clear()
    ax3.set_title('Bode plot Go')
    ax3.semilogx(omega, 20 * np.log10(mag), '-bD', markevery=[i_freq])
    ax3.set_ylim([-50,20])
    ax3.grid()
    
    ax4.clear()
    ax4.semilogx(omega, phase*180/np.pi, '-bD', markevery=[i_freq])
    ax4.grid()
    
slider.unobserve_all()
slider.observe(replot_bode, names=['value'])

fig = plt.figure()
fig.set_label(' ')

ax1 = fig.add_subplot(221, projection=None)
ax2 = fig.add_subplot(223, projection=None)
ax3 = fig.add_subplot(222, projection=None)
ax4 = fig.add_subplot(224, projection=None)

replot_bode()
slider

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

FloatSlider(value=1.0, description='Kr', min=1.0)